# first complete the remaining part of this notebook(create a proper rag)



# second assisgnment is: take a multiple pdf with text,image,table
1. fetch the data from pdf
2. at lesat there should be 200 pages
3. if chunking(use the sementic chumnking technique) required do chunking and then embedding
4. store it inside the vector database(use any of them 1. mongodb 2. astradb 3. opensearch 4.milvus) ## i have not discuss then you need to explore
5. create a index with all three index machnism(Flat, HNSW, IVF) ## i have not discuss then you need to explore
6. create a retriever pipeline
7. check the retriever time(which one is fastet)
8. print the accuray score of every similarity search
9. perform the reranking either using BM25 or MMR ## i have not discuss then you need to explore
10. then write a prompt template
11. generte a oputput through llm
12. render that output over the DOCx ## i have not discuss then you need to explore
as a additional tip: you can follow rag playlist from my youtube

after completing it keep it on your github and share that link on my  mail id:
snshrivas3365@gmail.com

and share the assignment in your community chat as well by tagging krish and sunny

deadline is: till firday 9PM
   
saturday
agentic session--> main session
langgraph

In [ ]:

from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.prompts import PromptTemplate
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import faiss
import os
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"

llm = ChatOpenAI(model="gpt-4.1-nano", api_key=OPENAI_API_KEY)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# Load and Split Docs
loader = PyPDFDirectoryLoader(path="pdfs")
docs = loader.load()
# text_splitter = SemanticChunker(
#     embeddings=embeddings,
#     breakpoint_threshold_type="percentile",
#     breakpoint_threshold_amount=85.0,
#     number_of_chunks=100,
#     min_chunk_size=3,
#     buffer_size=1
# )
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=70)
docs_splitted = text_splitter.split_documents(docs)
# VectorStore

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

# Add Docs
vector_store.add_documents(documents=docs_splitted)
# Similarity Search
vector_store.similarity_search_with_score("supervised fine tunning")
# Keywords Search
keyword_retriever = BM25Retriever.from_documents(docs_splitted)
keyword_retriever.k = 3
keyword_retriever.invoke("supervised fine tunning")
# Pipeline
similarity_retriever= vector_store.as_retriever(search_kwargs={"k": 3})

hybrid_search_retriever = EnsembleRetriever(retrievers=[similarity_retriever, keyword_retriever], weights=[0.3, 0.7])
hybrid_search_retriever.invoke("What is SFT(supervised fine tunning)?")
template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. Return the documents that you specically used for your answer. \nQuestion: {question} \nContext: {context} \nAnswer:")
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)
def format_docs(docs):
    return "/n/n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": hybrid_search_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
rag_chain.invoke("What is supervised fine tunning?")